#### Import Libraries

In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,  MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

#### 1. Load and Integrate Data

In [2]:
# Load datasets
movies = pd.read_csv('movies.csv')  
ratings = pd.read_csv('ratings.csv')   

In [3]:
# check top rows for movies
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
# check top rows for ratings
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [5]:
# check dataset shape
print("Shape of movies:", movies.shape)
print("Shape of ratings:", ratings.shape)

Shape of movies: (9742, 3)
Shape of ratings: (100836, 4)


In [6]:
# Check data type for movies
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
# Check data type for rating
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


#### 2. Data Cleaning

In [8]:
# Checking for missing values
print("Missing values in movies:")
print(movies.isnull().sum())
print("\nMissing values in ratings:")
print(ratings.isnull().sum())

Missing values in movies:
movieId    0
title      0
genres     0
dtype: int64

Missing values in ratings:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


* There are no missing rows in all the dataset

In [9]:
#split the movie genre
movies['genres'] = movies['genres'].str.split('|') 

In [10]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"


#### 3. Pre-processing and Feature Generation

##### a. Ratings

In [11]:
# extract datetime, hour, weekday
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings['hour'] = ratings['datetime'].dt.hour
ratings['weekday'] = ratings['datetime'].dt.dayofweek

In [12]:
ratings.head(2)

,userId,movieId,rating,timestamp,datetime,hour,weekday
0,1,1,4.0,964982703,2000-07-30 18:45:03,18,6
1,1,3,4.0,964981247,2000-07-30 18:20:47,18,6


##### b. Movies

In [13]:
# Generate Movie Features and Transform Genre
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(movies['genres']),
                              columns=mlb.classes_,
                              index=movies.index)

movies_feats = pd.concat([movies[['movieId','title']], genres_encoded], axis=1)

In [14]:
movies_feats.head(2)

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Merge Ratings, Movies, Tags
df = ratings.merge(movies_feats, on='movieId', how='left').fillna(0)

In [16]:
df.head()

,userId,movieId,rating,timestamp,datetime,hour,weekday,title,(no genres listed),Action,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,964982703,2000-07-30 18:45:03,18,6,Toy Story (1995),0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,2000-07-30 18:20:47,18,6,Grumpier Old Men (1995),0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,2000-07-30 18:37:04,18,6,Heat (1995),0,1,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,2000-07-30 19:03:35,19,6,Seven (a.k.a. Se7en) (1995),0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,2000-07-30 18:48:51,18,6,"Usual Suspects, The (1995)",0,0,...,0,0,0,0,1,0,0,1,0,0


In [18]:
# Create Features for User stats and movie stats
user_stats = df.groupby('userId')['rating'].agg(['mean', 'count'])\
               .rename(columns={'mean':'user_avg','count':'user_count'})
movie_stats = df.groupby('movieId')['rating'].agg(['mean', 'count'])\
                .rename(columns={'mean':'movie_avg','count':'movie_count'})

df = df.join(user_stats, on='userId').join(movie_stats, on='movieId')

##### 4. Standardization & Variance Filter

In [19]:
# columns selection
exclude_cols = ['userId','movieId','rating','timestamp','datetime']
feature_cols = [c for c in df.columns if c not in exclude_cols]

##### Normalization/Standardization

In [22]:
# Normalize features to [0,1]
scaler = MinMaxScaler()

# Identify numeric feature columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Exclude ID & target columns from numeric processing
numeric_feature_cols = [c for c in numeric_cols if c not in ['userId','movieId','rating','timestamp']]

# 1. Normalize numeric features
scaler = MinMaxScaler()
df[numeric_feature_cols] = scaler.fit_transform(df[numeric_feature_cols])

##### Variance Filter

In [ ]:
# Apply VarianceThreshold on numeric columns 
X_numeric = df[numeric_feature_cols].values
sel = VarianceThreshold(threshold=0.001)
X_reduced = sel.fit_transform(X_numeric)

In [25]:
# Keep selected feature names
selected_cols = [col for col, keep in zip(feature_cols, sel.get_support()) if keep]

In [26]:
# Build Final Cleaned and Reduced DataFrame
df_pruned = pd.concat([
    df[['userId','movieId','rating']],
    pd.DataFrame(X_reduced, columns=selected_cols, index=df.index)
], axis=1)

In [28]:
df_pruned.head()

,userId,movieId,rating,hour,weekday,(no genres listed),Action,Adventure,Animation,Children,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,user_avg,user_count,movie_avg
0,1,1,4.0,0.782609,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.8299,0.079164,0.760207,0.652439
1,1,3,4.0,0.782609,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.8299,0.079164,0.613248,0.155488
2,1,6,4.0,0.782609,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.8299,0.079164,0.765795,0.307927
3,1,47,5.0,0.826087,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.8299,0.079164,0.772304,0.615854
4,1,50,5.0,0.782609,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.8299,0.079164,0.830610,0.618902


In [27]:
# Save processed and cleaned datasets
df_pruned.to_csv('netflix_cleaned_data.csv', index=False)